In [1]:
"""
Feature: blood_culture_performed
Flag if blood culture was performed from microbiologyevents
"""
import pandas as pd
import os
from config_paths import *
from utils import drop_columns

In [2]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)

# Load microbiology events
microbiology_events = pd.read_csv(
    os.path.join(hosp_path, "microbiologyevents.csv"),
    low_memory=False
)

# Filter blood culture
blood_culture_df = microbiology_events[
    microbiology_events["spec_type_desc"].str.upper() == "BLOOD CULTURE"
].copy()

# Admission-level flag
blood_culture_flag = (
    blood_culture_df[["subject_id", "hadm_id"]]
    .drop_duplicates()
    .assign(blood_culture_performed=True)
)

# Merge
df = df.merge(blood_culture_flag, on=["subject_id", "hadm_id"], how="left")

# Convert to boolean (matching notebook: just .notna())
df["blood_culture_performed"] = df["blood_culture_performed"].notna()

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nFeature 'blood_culture_performed' added.")
print(f"Value counts:\n{df['blood_culture_performed'].value_counts()}")
print(f"Total rows: {len(df)}")
print(f"True count: {df['blood_culture_performed'].sum()}")
print(f"False count: {(~df['blood_culture_performed']).sum()}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")


Feature 'blood_culture_performed' added.
Value counts:
blood_culture_performed
False    108751
True      49269
Name: count, dtype: int64
Total rows: 158020
True count: 49269
False count: 108751
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 92)


In [3]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [4]:
# cols_to_drop = [
#  "blood_culture_performed"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)